In [ ]:
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import time
import random
import datetime
from threading import Thread
from collections import deque, defaultdict
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

class MLEnhancer:
    def __init__(self):
        # 初始化机器学习模型
        self.order_model = GradientBoostingRegressor(n_estimators=100)
        self.priority_model = GradientBoostingRegressor(n_estimators=50)
        self.scaler = MinMaxScaler()
        
        # 历史数据存储
        self.history_data = pd.DataFrame(columns=[
            'hour', 'minute', 'order_type', 'item_count', 
            'equipment_load', 'staff_busy', 'urgency'
        ])
        
        # 模型训练状态
        self.models_trained = False
        self.next_training = datetime.datetime.now()
    
    def 记录状态(self, simulator):
        """记录当前模拟器状态用于机器学习训练"""
        current_time = simulator.当前时间
        
        # 计算设备负载
        equipment_load = sum(
            eq['使用中']/eq['容量'] 
            for eq in simulator.设备.values() 
            if eq['容量'] > 0
        ) / len(simulator.设备)
        
        # 计算员工忙碌度
        staff_busy = sum(
            1 for staff in simulator.后厨员工.values() 
            if staff['当前任务'] is not None or staff['被动任务'] is not None
        ) / max(1, len(simulator.后厨员工))
        
        # 计算平均订单内容数量
        all_orders = simulator.外卖订单 + simulator.堂食订单
        item_count = np.mean([len(o['内容']) for o in all_orders]) if all_orders else 0
        
        # 创建新数据行
        new_row = {
            'hour': current_time.hour,
            'minute': current_time.minute,
            'order_type': '外卖' if random.random() < 0.5 else '堂食',
            'item_count': item_count,
            'equipment_load': equipment_load,
            'staff_busy': staff_busy,
            'urgency': len(simulator.任务队列) / max(1, 50)
        }
        
        # 添加到历史数据
        # 修改为使用 loc 直接添加新行:
        if len(self.history_data) == 0:
            # 如果是空的 DataFrame，直接创建
            self.history_data = pd.DataFrame([new_row])
        else:
            # 否则添加一行
            new_idx = len(self.history_data)
            self.history_data.loc[new_idx] = new_row
        
        # 定期训练模型
        if self.history_data.shape[0] > 100 and datetime.datetime.now() > self.next_training:
            self._train_models()
            self.next_training = datetime.datetime.now() + datetime.timedelta(minutes=5)
    
    def _train_models(self):
        """训练预测模型和优先级模型"""
        if self.history_data.shape[0] < 100:
            return
        
        try:
            # 训练订单预测模型
            X_order = self.history_data[['hour', 'minute', 'equipment_load', 'staff_busy']]
            y_order = self.history_data['item_count']
            self.order_model.fit(X_order, y_order)
            
            # 训练优先级模型（使用最近100条数据）
            recent_data = self.history_data.tail(100)
            X_priority = recent_data[['equipment_load', 'staff_busy', 'urgency']]
            y_priority = recent_data['item_count']  # 用订单数作为优先级指标
            self.priority_model.fit(X_priority, y_priority)
            
            self.models_trained = True
        except Exception as e:
            print(f"模型训练错误: {str(e)}")
    
    def 预测订单高峰(self, simulator):
        """预测未来30分钟的订单量"""
        if not self.models_trained or self.history_data.shape[0] < 100:
            return None
        
        try:
            # 准备当前状态数据
            current_time = simulator.当前时间
            current_data = pd.DataFrame([{
                'hour': current_time.hour, 
                'minute': current_time.minute,
                'equipment_load': sum(eq['使用中']/eq['容量'] for eq in simulator.设备.values() if eq['容量'] > 0) / len(simulator.设备),
                'staff_busy': sum(1 for staff in simulator.后厨员工.values() if staff['当前任务'] is not None) / max(1, len(simulator.后厨员工))
            }])
            
            # 预测未来30分钟的状态（创建30个时间点）
            future_times = []
            for i in range(1, 31):
                future_time = current_time + datetime.timedelta(minutes=i)
                future_times.append({
                    'hour': future_time.hour,
                    'minute': future_time.minute,
                    'equipment_load': current_data['equipment_load'].values[0],  # 使用当前负载作为基础
                    'staff_busy': current_data['staff_busy'].values[0]  # 使用当前忙碌度作为基础
                })
            
            future_data = pd.DataFrame(future_times)
            
            # 预测未来订单量
            predictions = self.order_model.predict(future_data)
            avg_prediction = np.mean(predictions)
            
            # 返回预测的平均订单数（非负值）
            return max(0, avg_prediction)
        except Exception as e:
            print(f"预测错误: {str(e)}")
            return None
    
    def 动态调整优先级(self, 任务队列, simulator):
        """根据模型调整任务优先级"""
        if not 任务队列:
            return []

        # 如果模型未训练，使用基本排序
        if not self.models_trained:
            return sorted(任务队列, key=lambda x: (
                0 if x.get('订单ID', 0) == 0 else 1,  # 预制库存任务最低优先级
                x.get('优先级', 3),  # 按原优先级
                -(datetime.datetime.now() - datetime.datetime.now()).total_seconds()  # 等待时间最长的优先
            ))

        try:
            # 计算当前系统负载
            equipment_load = sum(eq['使用中']/eq['容量'] for eq in simulator.设备.values() if eq['容量'] > 0) / len(simulator.设备)
            staff_busy = sum(1 for staff in simulator.后厨员工.values() if staff['当前任务'] is not None) / max(1, len(simulator.后厨员工))

            # 为每个任务计算特征
            task_features = []
            task_ids = []
            for idx, 任务 in enumerate(任务队列):
                # 基本优先级
                基本优先级 = 任务.get('优先级', 3)

                # 计算等待时间
                等待时间 = 0
                订单ID = 任务.get('订单ID', 0)
                if 订单ID > 0:
                    for 订单列表 in [simulator.外卖订单, simulator.堂食订单]:
                        for 订单 in 订单列表:
                            if 订单['ID'] == 订单ID:
                                等待时间 = (simulator.当前时间 - 订单['创建时间']).total_seconds() / 60
                                break

                # 批量处理数量
                批量数 = len(任务.get('待炸数量', [])) if 任务.get('待炸数量') else 1

                # 只使用稳定的特征
                task_features.append([
                    基本优先级,
                    等待时间,
                    批量数
                ])
                task_ids.append(idx)

            if not task_features:
                return 任务队列

            # 安全地标准化特征
            try:
                scaled_features = self.scaler.fit_transform(task_features)
            except Exception as e:
                print(f"特征缩放错误: {str(e)}")
                # 出错时使用原始特征
                scaled_features = np.array(task_features)

            # 为每个任务计算最终优先级
            task_priorities = []
            for i, 任务 in enumerate(任务队列):
                # 基本优先级
                base_priority = 任务.get('优先级', 3)

                # 等待时间加成
                等待时间 = task_features[i][1] if i < len(task_features) else 0
                waiting_boost = min(2.0, 1.0 + 等待时间 / 50.0)

                # 批量处理加成
                批量数 = task_features[i][2] if i < len(task_features) else 1
                batch_boost = 1.0 + (批量数 - 1) * 0.05

                # 订单类型加成
                type_boost = 1.5 if base_priority == 1 else 1.0

                # 计算最终优先级分数
                final_priority = (10 - base_priority) * waiting_boost * batch_boost * type_boost

                task_priorities.append((i, final_priority))

            # 按最终优先级降序排序
            sorted_indices = [idx for idx, _ in sorted(task_priorities, key=lambda x: x[1], reverse=True)]

            # 返回排序后的任务队列
            return [任务队列[i] for i in sorted_indices]

        except Exception as e:
            print(f"优先级调整错误: {str(e)}")
            # 出错时返回原任务队列
            return 任务队列
            
            # 标准化特征
            scaled_features = self.scaler.fit_transform(task_features)
            
            # 用优先级模型预测调整系数
            boost_factors = np.zeros(len(任务队列))
            
            # 只对有特征的任务进行预测
            if len(task_ids) > 0:
                # 提取用于预测的特征子集
                prediction_features = scaled_features[:, [0, 1, 4]]  # 使用优先级、等待时间和批量数
                model_input = np.column_stack((prediction_features, np.ones(len(task_ids))))
                
                # 预测调整系数
                predicted_factors = self.priority_model.predict(model_input[:, [4, 5]])
                
                # 应用预测调整到对应任务
                for i, idx in enumerate(task_ids):
                    boost_factors[idx] = predicted_factors[i]
            
            # 计算最终优先级并排序
            task_priorities = []
            for i, 任务 in enumerate(任务队列):
                # 基本优先级（1是最高）
                base_priority = 任务.get('优先级', 3)
                
                # 等待时间加成：每等待5分钟增加10%权重
                等待时间 = 0
                订单ID = 任务.get('订单ID', 0)
                if 订单ID > 0:
                    for 订单列表 in [simulator.外卖订单, simulator.堂食订单]:
                        for 订单 in 订单列表:
                            if 订单['ID'] == 订单ID:
                                等待时间 = (simulator.当前时间 - 订单['创建时间']).total_seconds() / 60
                                break
                
                waiting_boost = min(2.0, 1.0 + 等待时间 / 50.0)  # 最多翻倍优先级
                
                # 批量处理加成：每多处理一个订单增加5%权重
                batch_size = len(任务.get('待炸数量', [])) if 任务.get('待炸数量') else 1
                batch_boost = 1.0 + (batch_size - 1) * 0.05  # 批量处理提升
                
                # 订单类型加成：堂食(优先级1)比外卖(优先级2)高50%权重
                type_boost = 1.5 if base_priority == 1 else 1.0
                
                # 模型预测加成
                model_boost = 1.0 + max(0, boost_factors[i])
                
                # 最终优先级分数（越大越优先）
                final_priority = (10 - base_priority) * waiting_boost * batch_boost * type_boost * model_boost
                
                task_priorities.append((i, final_priority))
            
            # 按最终优先级降序排序
            sorted_indices = [idx for idx, _ in sorted(task_priorities, key=lambda x: x[1], reverse=True)]
            
            # 返回排序后的任务队列
            return [任务队列[i] for i in sorted_indices]
        
        except Exception as e:
            print(f"优先级调整错误: {str(e)}")
            # 出错时返回原任务队列
            return 任务队列

class 餐厅模拟器:
    def __init__(self):
        # 时间系统
        self.当前时间 = datetime.datetime.combine(datetime.date.today(), datetime.time(9, 0))
        self.结束时间 = datetime.datetime.combine(datetime.date.today(), datetime.time(22, 0))
        self.速度 = 1
        self.暂停 = True
        self.日志 = []
        
        # 添加炸篮容量配置
        self.炸篮容量 = {
            '薯条': 6,
            '鸡块': 8,
            '炸鸡腿': 20,
            '炸鸡翅': 12
        }
        
        # 统计数据
        self.超时订单数 = 0
        self.严重超时订单数 = 0
        self.不高兴顾客数 = 0
        self.生气顾客数 = 0
        
        # 订单系统
        self.外卖订单 = []
        self.堂食订单 = []
        self.完成订单 = []
        self.订单ID = 1
        self.座位数 = 20
        self.当前座位使用 = 0
        
        # 设备系统
        self.设备 = {
            '低温炸炉': {'容量': 8, '使用中': 0, '任务': []},
            '高温炸炉': {'容量': 8, '使用中': 0, '任务': []},
            '煎台': {'容量': 10, '使用中': 0, '任务': []},
            '保温箱': {'容量': 50, '使用中': 0, '内容': []},
            '可乐机': {'容量': 2, '使用中': 0, '任务': []},
            '冰淇淋机': {'容量': 1, '使用中': 0, '任务': []}
        }
        
        # 新增炸篮系统
        self.炸篮 = {'总数': 8, '空闲': 8, '使用中': []}
        
        # 库存系统 - 预制食品
        self.库存 = {
            '汉堡A': {'数量': 0, '过期时间': None},
            '汉堡B': {'数量': 0, '过期时间': None},
            '薯条': {'数量': 0, '过期时间': None},
            '炸鸡腿': {'数量': 0, '过期时间': None},
            '炸鸡翅': {'数量': 0, '过期时间': None},
            '炸鸡块': {'数量': 0, '过期时间': None},
            '可乐': {'数量': 0, '过期时间': None},
            '冰淇淋': {'数量': 0, '过期时间': None},  # 添加冰淇淋到库存，即使它不存储
            '预炸薯条': {'数量': 0, '过期时间': None},
            '预炸鸡腿': {'数量': 0, '过期时间': None}
        }
        
        # 员工系统 - 分为前台和后厨
        self.前台员工 = {1: {'当前任务': None, '结束时间': None, '被动任务': None, '被动结束时间': None}}
        self.后厨员工 = {}
        self.更新员工数量()
        
        # 任务队列
        self.任务队列 = []
        
        # 任务配方
        self.产品配方 = {
            '汉堡A': [
                {'名称': '煎肉饼', '设备': '煎台', '时间': 120, '类型': '后厨', '需要监督': True},
                {'名称': '组装汉堡A', '设备': None, '时间': 60, '类型': '后厨', '需要监督': True}
            ],
            '汉堡B': [
                {'名称': '炸鸡腿排', '设备': '高温炸炉', '时间': 270, '类型': '后厨', '需要炸篮': True, '需要监督': False, '食品类型': '炸鸡腿'},
                {'名称': '组装汉堡B', '设备': None, '时间': 60, '类型': '后厨', '需要监督': True}
            ],
            '薯条': [
                {'名称': '预炸薯条', '设备': '低温炸炉', '时间': 240, '类型': '后厨', '需要炸篮': True, '需要监督': False, '食品类型': '薯条'},
                {'名称': '复炸薯条', '设备': '高温炸炉', '时间': 165, '类型': '后厨', '需要炸篮': True, '需要监督': False, '食品类型': '薯条'}
            ],
            '炸鸡腿': [
                {'名称': '预炸鸡腿', '设备': '低温炸炉', '时间': 420, '类型': '后厨', '需要炸篮': True, '需要监督': False, '食品类型': '炸鸡腿'},  # 7分钟
                {'名称': '复炸鸡腿', '设备': '高温炸炉', '时间': 300, '类型': '后厨', '需要炸篮': True, '需要监督': False, '食品类型': '炸鸡腿'}   # 5分钟
            ],
            '炸鸡翅': [
                {'名称': '炸鸡翅', '设备': '高温炸炉', '时间': 360, '类型': '后厨', '需要炸篮': True, '需要监督': False, '食品类型': '炸鸡翅'}
            ],
            '炸鸡块': [
                {'名称': '炸鸡块', '设备': '高温炸炉', '时间': 210, '类型': '后厨', '需要炸篮': True, '需要监督': False, '食品类型': '鸡块'}
            ],
            '可乐': [
                {'名称': '接可乐', '设备': '可乐机', '时间': 30, '类型': '前台', '需要监督': True}
            ],
            '冰淇淋': [
                {'名称': '做冰淇淋', '设备': '冰淇淋机', '时间': 60, '类型': '前台', '需要监督': True}
            ]
        }
        
        # 保存时间
        self.保存时间 = {
            '汉堡A': 1800,  # 30分钟
            '汉堡B': 1800,  # 30分钟
            '薯条': 3600,   # 1小时
            '炸鸡腿': 7200,  # 2小时
            '炸鸡翅': 7200,  # 2小时
            '炸鸡块': 7200,  # 2小时
            '可乐': 3600,    # 1小时
            '预炸薯条': 10800,  # 3小时
            '预炸鸡腿': 10800   # 3小时
        }
        
        # 套餐配置
        self.套餐 = {
            '单人套餐A': {
                '内容': ['汉堡A', '薯条', '炸鸡块', '可乐'],
                '外卖概率': 0.20,
                '堂食概率': 0.15
            },
            '单人套餐B': {
                '内容': ['汉堡B', '薯条', '炸鸡翅', '可乐'],
                '外卖概率': 0.20,
                '堂食概率': 0.15
            },
            '单人套餐C': {
                '内容': ['汉堡B', '薯条', '可乐'],
                '外卖概率': 0.25,
                '堂食概率': 0.20
            },
            '双人套餐A': {
                '内容': ['汉堡A', '汉堡B', '薯条', '炸鸡翅', '炸鸡腿', '可乐', '可乐'],
                '外卖概率': 0.10,
                '堂食概率': 0.15,
                '座位数': 2
            },
            '双人套餐B': {
                '内容': ['汉堡A', '汉堡B', '薯条', '薯条', '炸鸡翅', '炸鸡块', '可乐', '可乐'],
                '外卖概率': 0.05,
                '堂食概率': 0.10,
                '座位数': 2
            },
            '单人套餐T': {
                '内容': ['汉堡A', '薯条', '可乐', '冰淇淋'],
                '外卖概率': 0.00,
                '堂食概率': 0.15
            }
        }
        
        # 初始化机器学习增强器
        self.ml = MLEnhancer()
        
        # 智能批处理系统相关参数
        self.上次智能预测时间 = self.当前时间
        self.智能预测周期 = 300  # 5分钟预测一次
    
    def 更新员工数量(self):
        # 根据时间段动态调整后厨员工数量
        小时 = self.当前时间.hour
        分钟 = self.当前时间.minute
        当前时刻 = 小时 + 分钟/60
        
        # 高峰期：11:30-12:30; 17:00-19:00
        if (11.5 <= 当前时刻 < 12.5) or (17 <= 当前时刻 < 19):
            员工数 = 3
        # 次高峰期：10:30-11:30; 12:30-13:30; 16:00-17:00; 19:00-20:00
        elif (10.5 <= 当前时刻 < 11.5) or (12.5 <= 当前时刻 < 13.5) or \
             (16 <= 当前时刻 < 17) or (19 <= 当前时刻 < 20):
            员工数 = 2
        # 非高峰期：9:00-10:30; 13:30-16:00; 20:00-22:00
        else:
            员工数 = 1
        
        # 更新后厨员工列表
        新员工 = {}
        for i in range(1, 员工数+1):
            if i in self.后厨员工:
                新员工[i] = self.后厨员工[i]
            else:
                新员工[i] = {'当前任务': None, '结束时间': None, '被动任务': None, '被动结束时间': None}
        
        # 移除过多的员工
        for i in list(self.后厨员工.keys()):
            if i not in 新员工:
                if self.后厨员工[i]['当前任务']:
                    self.添加日志(f"后厨员工{i}下班，未完成的任务回到队列：{self.后厨员工[i]['当前任务']['名称']}")
                    self.任务队列.append(self.后厨员工[i]['当前任务'])
                if self.后厨员工[i]['被动任务']:
                    self.添加日志(f"后厨员工{i}下班，被动任务'{self.后厨员工[i]['被动任务']['名称']}'交接给其他员工")
                    # 释放炸篮
                    if self.后厨员工[i]['被动任务'].get('需要炸篮'):
                        for idx, (员工类型, 员工id, _) in enumerate(self.炸篮['使用中']):
                            if 员工类型 == '后厨' and 员工id == i:
                                self.炸篮['使用中'].pop(idx)
                                self.炸篮['空闲'] += 1
                                break
                    # 释放设备
                    if self.后厨员工[i]['被动任务']['设备']:
                        设备名 = self.后厨员工[i]['被动任务']['设备']
                        self.设备[设备名]['使用中'] -= 1
                        for idx, (类型, id, 任务) in enumerate(self.设备[设备名]['任务']):
                            if 类型 == '后厨' and id == i:
                                self.设备[设备名]['任务'].pop(idx)
                                break
        
        self.后厨员工 = 新员工
    
    def 生成订单概率(self, 类型):
        # 根据时间段生成订单概率
        小时 = self.当前时间.hour
        分钟 = self.当前时间.minute
        当前时刻 = 小时 + 分钟/60
        
        # 高峰期：11:30-12:30; 17:00-19:00
        if (11.5 <= 当前时刻 < 12.5) or (17 <= 当前时刻 < 19):
            if 类型 == '外卖':
                return 30/60/60  # 每秒概率 (每小时30单)
            else:  # 堂食
                return 1.0  # 100%上座率
        # 次高峰期：10:30-11:30; 12:30-13:30; 16:00-17:00; 19:00-20:00
        elif (10.5 <= 当前时刻 < 11.5) or (12.5 <= 当前时刻 < 13.5) or \
             (16 <= 当前时刻 < 17) or (19 <= 当前时刻 < 20):
            if 类型 == '外卖':
                return 20/60/60  # 每秒概率 (每小时20单)
            else:  # 堂食
                return 0.5  # 50%上座率
        # 非高峰期：9:00-10:30; 13:30-16:00; 20:00-22:00
        else:
            if 类型 == '外卖':
                return 10/60/60  # 每秒概率 (每小时10单)
            else:  # 堂食
                return 0.2  # 20%上座率
    
    def 生成随机套餐(self, 类型):
        总概率 = 0
        套餐列表 = []
        
        # 根据类型获取对应概率
        for 名称, 信息 in self.套餐.items():
            if 类型 == '外卖':
                概率 = 信息['外卖概率']
            else:
                概率 = 信息['堂食概率']
            
            if 概率 > 0:
                总概率 += 概率
                套餐列表.append((名称, 概率))
        
        # 标准化概率并选择
        r = random.random() * 总概率
        累计 = 0
        for 名称, 概率 in 套餐列表:
            累计 += 概率
            if r <= 累计:
                return 名称
        
        # 随机生成非套餐订单 (20%概率)
        商品列表 = []
        # 随机1-2个汉堡
        汉堡数 = random.randint(1, 2)
        for _ in range(汉堡数):
            商品列表.append(random.choice(['汉堡A', '汉堡B']))
        
        # 随机0-1个其他单品
        其他单品 = ['薯条', '炸鸡腿', '炸鸡翅', '炸鸡块', '可乐']
        if 类型 == '堂食':
            其他单品.append('冰淇淋')
        
        随机单品数 = random.randint(0, min(len(其他单品), 3))
        商品列表.extend(random.sample(其他单品, 随机单品数))
        
        return {'自定义': 商品列表}
    
    def 添加订单(self, 类型):
        # 生成一个新订单
        套餐名称 = self.生成随机套餐(类型)
        
        # 处理套餐内容
        if isinstance(套餐名称, dict) and '自定义' in 套餐名称:
            内容 = 套餐名称['自定义']
            套餐名称 = '自定义套餐'
            座位数 = 1
        else:
            内容 = self.套餐[套餐名称]['内容'].copy()
            座位数 = self.套餐[套餐名称].get('座位数', 1) if 类型 == '堂食' else 1
        
        # 创建订单
        订单 = {
            'ID': self.订单ID,
            '类型': 类型,
            '套餐': 套餐名称,
            '内容': 内容,
            '创建时间': self.当前时间,
            '状态': '等待中',
            '完成时间': None,
            '座位数': 座位数,
            '离开时间': None if 类型 == '外卖' else self.当前时间 + datetime.timedelta(minutes=10),
            '顾客状态': '正常' if 类型 == '堂食' else None
        }
        
        self.订单ID += 1
        
        # 添加到对应队列
        if 类型 == '外卖':
            self.外卖订单.append(订单)
            self.添加日志(f"新外卖订单 #{订单['ID']}: {套餐名称}")
        else:
            if self.当前座位使用 + 座位数 <= self.座位数:
                self.堂食订单.append(订单)
                self.当前座位使用 += 座位数
                self.添加日志(f"新堂食订单 #{订单['ID']}: {套餐名称}，占用{座位数}个座位")
        
        # 分解为任务
        self.创建订单任务(订单)
    
    def 创建订单任务(self, 订单):
        任务列表 = []
        已用产品 = set()

        # 检查库存是否有现成品
        for 产品 in 订单['内容']:
            if 产品 in self.库存 and self.库存[产品]['数量'] > 0 and self.库存[产品]['过期时间'] and self.库存[产品]['过期时间'] > self.当前时间:
                self.库存[产品]['数量'] -= 1

                # 如果是可乐，不需要从保温箱中取
                if 产品 != '可乐':
                    # 从保温箱中移除对应食品
                    for idx, 内容项 in enumerate(self.设备['保温箱']['内容']):
                        if 内容项['产品'] == 产品 and 内容项['数量'] > 0:
                            内容项['数量'] -= 1
                            self.设备['保温箱']['使用中'] -= 1

                            # 如果该批次的食品已全部使用，移除此项
                            if 内容项['数量'] == 0:
                                self.设备['保温箱']['内容'].pop(idx)
                            break

                    self.添加日志(f"从保温箱取出 {产品} 来完成订单 #{订单['ID']}")
                else:
                    self.添加日志(f"从饮料柜取出 {产品} 来完成订单 #{订单['ID']}")

                已用产品.add(产品)
                continue

            # 需要制作的产品
            if 产品 not in 已用产品:
                for 步骤 in self.产品配方.get(产品, []):
                    任务 = 步骤.copy()
                    任务.update({
                        '订单ID': 订单['ID'],
                        '状态': '等待中',
                        '产品': 产品,
                        '优先级': 1 if 订单['类型'] == '堂食' else 2
                    })
                    任务列表.append(任务)

        # 若订单内容全部从库存获取，则直接完成订单
        if len(已用产品) == len(订单['内容']):
            订单['状态'] = '已完成'
            订单['完成时间'] = self.当前时间
            if 订单['类型'] == '外卖':
                self.外卖订单.remove(订单)
            else:
                self.堂食订单.remove(订单)
                # 堂食客人会在离开时间后释放座位
            self.完成订单.append(订单)
            self.添加日志(f"订单 #{订单['ID']} 已从库存完成")
            return

        # 添加到任务队列
        self.任务队列.extend(任务列表)
        订单['状态'] = '处理中'
    
    def 检查员工可用性(self, 员工类型, 员工ID, 任务):
        """检查员工是否可以承担新任务"""
        if 员工类型 == '前台':
            员工 = self.前台员工[员工ID]
        else:
            员工 = self.后厨员工[员工ID]
        
        # 如果员工当前没有主要任务，或者正在处理不需要监督的被动任务
        if 员工['当前任务'] is None:
            return True
        
        # 如果当前有主动任务且需要监督，无法接新任务
        if 员工['当前任务'] and 员工['当前任务'].get('需要监督', True):
            return False
        
        # 如果当前任务是被动等待且新任务需要监督，检查是否会冲突
        if 任务.get('需要监督', True) and 员工['被动任务']:
            # 计算两个任务的剩余时间
            被动剩余时间 = (员工['被动结束时间'] - self.当前时间).total_seconds()
            # 如果被动任务即将完成，不要接新的需要监督的任务
            if 被动剩余时间 < 30:  # 30秒警戒线
                return False
        
        return True
    
    def 检查并合并炸锅任务(self, 任务):
        """尝试将炸锅任务合并到现有任务中"""
        if not 任务['设备'] or 任务['设备'] not in ['低温炸炉', '高温炸炉']:
            return False

        食品类型 = 任务.get('食品类型')
        if not 食品类型 or 食品类型 not in self.炸篮容量:
            return False

        # 检查现有任务是否可以合并
        for 设备任务 in self.设备[任务['设备']]['任务']:
            类型, id, 现有任务 = 设备任务
            if (现有任务.get('食品类型') == 食品类型 and 
                len(现有任务.get('待炸数量', [])) < self.炸篮容量[食品类型]):
                # 可以合并到现有炸篮
                if '待炸数量' not in 现有任务:
                    现有任务['待炸数量'] = [现有任务['订单ID']]
                现有任务['待炸数量'].append(任务['订单ID'])
                self.添加日志(f"合并订单#{任务['订单ID']}的{食品类型}到订单#{现有任务['订单ID']}的炸篮中")
                return True
        return False
    
    def 分配任务(self):
        # 使用机器学习动态调整任务优先级
        self.任务队列 = self.ml.动态调整优先级(self.任务队列, self)
        
        # 优先处理等待中的任务
        if not self.任务队列:
            return
        
        # 尝试分配任务给前台员工
        前台任务 = [i for i, 任务 in enumerate(self.任务队列) if 任务['类型'] == '前台']
        for 任务索引 in 前台任务:
            if 任务索引 >= len(self.任务队列):
                continue
            任务 = self.任务队列[任务索引]
            for 员工ID in self.前台员工:
                if self.检查员工可用性('前台', 员工ID, 任务):
                    # 检查设备是否可用
                    if 任务['设备'] and self.设备[任务['设备']]['使用中'] >= self.设备[任务['设备']]['容量']:
                        continue
                    
                    # 分配任务给员工
                    self.前台员工[员工ID]['当前任务'] = 任务
                    self.前台员工[员工ID]['开始时间'] = self.当前时间
                    self.前台员工[员工ID]['结束时间'] = self.当前时间 + datetime.timedelta(seconds=任务['时间'])
                    
                    # 更新设备状态
                    if 任务['设备']:
                        self.设备[任务['设备']]['使用中'] += 1
                        self.设备[任务['设备']]['任务'].append(('前台', 员工ID, 任务))
                    
                    self.任务队列.pop(任务索引)
                    self.添加日志(f"前台员工{员工ID}开始{任务['名称']}，预计{任务['时间']//60}分{任务['时间']%60}秒后完成")
                    return  # 成功分配一个任务后返回，下次循环继续
        
        # 智能合并炸锅任务
        self.智能合并炸篮任务()
        
        # 尝试分配任务给后厨员工
        后厨任务索引列表 = [i for i, 任务 in enumerate(self.任务队列) if 任务['类型'] == '后厨']
        for 任务索引 in 后厨任务索引列表:
            if 任务索引 >= len(self.任务队列):  # 添加索引检查
                continue
            任务 = self.任务队列[任务索引]

            # 如果是炸锅任务，尝试合并
            if 任务.get('设备') in ['低温炸炉', '高温炸炉']:
                食品类型 = 任务.get('食品类型')
                if 食品类型 in self.炸篮容量:
                    # 如果可以合并到现有任务中
                    if self.检查并合并炸锅任务(任务):
                        # 注意：由于删除了一个元素，后面的索引都会变化，所以直接返回重新开始
                        self.任务队列.pop(任务索引)
                        return
                    # 如果是新任务，初始化待炸数量
                    任务['待炸数量'] = [任务['订单ID']]

            # 检查炸篮总数限制（如果任务需要炸篮）
            当前使用的炸篮总数 = (self.设备['低温炸炉']['使用中'] + self.设备['高温炸炉']['使用中'])
            if 任务.get('需要炸篮', False) and 当前使用的炸篮总数 >= 8:
                continue
            
            for 员工ID in self.后厨员工:
                if self.检查员工可用性('后厨', 员工ID, 任务):
                    # 检查设备是否可用
                    if 任务['设备'] and self.设备[任务['设备']]['使用中'] >= self.设备[任务['设备']]['容量']:
                        continue
                    
                    # 分配任务类型（主动或被动）
                    if 任务.get('需要监督', True):
                        # 需要监督的任务作为主动任务
                        self.后厨员工[员工ID]['当前任务'] = 任务
                        self.后厨员工[员工ID]['开始时间'] = self.当前时间
                        self.后厨员工[员工ID]['结束时间'] = self.当前时间 + datetime.timedelta(seconds=任务['时间'])
                        任务类型 = "主动"
                    else:
                        # 不需要监督的任务（如炸制）作为被动任务
                        self.后厨员工[员工ID]['被动任务'] = 任务
                        self.后厨员工[员工ID]['被动开始时间'] = self.当前时间
                        self.后厨员工[员工ID]['被动结束时间'] = self.当前时间 + datetime.timedelta(seconds=任务['时间'])
                        任务类型 = "被动"
                    
                    # 更新设备状态
                    if 任务['设备']:
                        self.设备[任务['设备']]['使用中'] += 1
                        self.设备[任务['设备']]['任务'].append(('后厨', 员工ID, 任务))
                    
                    # 更新炸篮状态
                    if 任务.get('需要炸篮', False):
                        self.炸篮['空闲'] -= 1
                        self.炸篮['使用中'].append(('后厨', 员工ID, 任务))
                    
                    self.任务队列.pop(任务索引)
                    self.添加日志(f"后厨员工{员工ID}开始{任务类型}任务:{任务['名称']}，预计{任务['时间']//60}分{任务['时间']%60}秒后完成")
                    return  # 成功分配一个任务后返回，下次循环继续
    
    def 智能合并炸篮任务(self):
        """智能合并同类型的炸锅任务以提高效率"""
        炸炉设备 = ['低温炸炉', '高温炸炉']
        
        # 计算当前炸炉负载
        current_load = sum(
            self.设备[eq]['使用中'] / self.设备[eq]['容量'] 
            for eq in 炸炉设备
        ) / len(炸炉设备)
        
        # 当炸炉负载较低时，更积极地合并任务
        合并阈值 = 0.7 - current_load * 0.3  # 负载越高，阈值越低
        
        # 先按食品类型分组任务
        任务分组 = defaultdict(list)
        for i, 任务 in enumerate(self.任务队列):
            if 任务.get('设备') in 炸炉设备 and 任务.get('食品类型'):
                任务分组[任务['食品类型']].append((i, 任务))
        
        # 对于每个食品类型，检查是否可以合并
        需移除索引 = []
        for 食品类型, 任务列表 in 任务分组.items():
            if len(任务列表) <= 1:
                continue
                
            # 排序：优先合并堂食订单和等待时间长的订单
            排序后任务 = sorted(任务列表, key=lambda x: (
                x[1].get('优先级', 3),  # 优先级低的先处理
                x[0]  # 保持索引顺序
            ))
            
            # 选择第一个任务作为基准任务
            基准索引, 基准任务 = 排序后任务[0]
            合并次数 = 0
            
            # 检查是否有待炸数量字段，没有则初始化
            if '待炸数量' not in 基准任务:
                基准任务['待炸数量'] = [基准任务['订单ID']]
            
            # 尝试合并其他任务
            for 索引, 任务 in 排序后任务[1:]:
                # 检查是否已经达到炸篮容量上限
                if len(基准任务['待炸数量']) >= self.炸篮容量[食品类型]:
                    break
                    
                # 合并任务
                基准任务['待炸数量'].append(任务['订单ID'])
                需移除索引.append(索引)
                合并次数 += 1
                self.添加日志(f"智能批处理: 合并订单#{任务['订单ID']}的{食品类型}到订单#{基准任务['订单ID']}的炸篮中")
                
                # 如果已经合并了足够多的任务，停止合并
                if 合并次数 >= 3:  # 最多合并3个任务
                    break
        
        # 从任务队列中移除已合并的任务（从大到小移除，避免索引错位）
        for 索引 in sorted(需移除索引, reverse=True):
            if 索引 < len(self.任务队列):
                self.任务队列.pop(索引)
    
    def 更新订单状态(self, 订单ID, 产品类型):
        """更新订单状态，用于批量完成炸篮任务"""
        # 查找对应订单
        for 订单列表 in [self.外卖订单, self.堂食订单]:
            for 订单 in 订单列表:
                if 订单['ID'] == 订单ID:
                    # 检查是否还有其他任务
                    self.添加日志(f"订单 #{订单ID} 的 {产品类型} 已完成")
                    return
    
    def 完成任务(self, 员工类型, 员工ID, 任务类型='当前任务'):
        if 员工类型 == '前台':
            员工 = self.前台员工[员工ID]
        else:
            员工 = self.后厨员工[员工ID]
        
        if 任务类型 == '当前任务':
            任务 = 员工['当前任务']
        else:
            任务 = 员工['被动任务']
        
        if not 任务:
            return
        
        产品 = 任务['产品']
        订单ID = 任务['订单ID']
        
        # 批量处理炸篮中的订单
        if 任务['设备'] in ['低温炸炉', '高温炸炉']:
            # 批量完成所有在同一炸篮中的订单
            for 订单ID in 任务.get('待炸数量', [任务['订单ID']]):
                # 更新订单状态
                self.更新订单状态(订单ID, 任务.get('食品类型', '食品'))
        
        # 释放设备
        if 任务['设备']:
            self.设备[任务['设备']]['使用中'] -= 1
            # 移除该任务
            for i, (类型, id, t) in enumerate(self.设备[任务['设备']]['任务']):
                if 类型 == 员工类型 and id == 员工ID and t['名称'] == 任务['名称']:
                    self.设备[任务['设备']]['任务'].pop(i)
                    break
        
        # 释放炸篮
        if 任务.get('需要炸篮', False):
            for i, (类型, id, t) in enumerate(self.炸篮['使用中']):
                if 类型 == 员工类型 and id == 员工ID and t['名称'] == 任务['名称']:
                    self.炸篮['使用中'].pop(i)
                    self.炸篮['空闲'] += 1
                    break
        
        self.添加日志(f"{员工类型}员工{员工ID}完成了{任务['名称']}（订单#{订单ID}）")
        
        # 处理特殊任务：准备食材相关
        if 任务['名称'] == '组装汉堡A' or 任务['名称'] == '组装汉堡B':
            产品名称 = 'A' if 任务['名称'] == '组装汉堡A' else 'B'
            self.添加库存('汉堡' + 产品名称, 1)
            
        elif 任务['名称'] == '复炸薯条':
            待炸数量 = len(任务.get('待炸数量', [任务['订单ID']]))
            self.添加库存('薯条', 待炸数量)
            
        elif 任务['名称'] == '复炸鸡腿':
            待炸数量 = len(任务.get('待炸数量', [任务['订单ID']]))
            self.添加库存('炸鸡腿', 待炸数量)
            
        elif 任务['名称'] == '炸鸡翅':
            待炸数量 = len(任务.get('待炸数量', [任务['订单ID']]))
            self.添加库存('炸鸡翅', 待炸数量)
            
        elif 任务['名称'] == '炸鸡块':
            待炸数量 = len(任务.get('待炸数量', [任务['订单ID']]))
            self.添加库存('炸鸡块', 待炸数量)
            
        elif 任务['名称'] == '接可乐':
            self.添加库存('可乐', 1)
            
        elif 任务['名称'] == '做冰淇淋':
            # 冰淇淋不保存，直接消费（不添加库存）
            self.添加日志(f"冰淇淋制作完成，直接提供给客人")
            
        elif 任务['名称'] == '预炸薯条':
            待炸数量 = len(任务.get('待炸数量', [任务['订单ID']]))
            self.添加库存('预炸薯条', 待炸数量)
            
        elif 任务['名称'] == '预炸鸡腿':
            待炸数量 = len(任务.get('待炸数量', [任务['订单ID']]))
            self.添加库存('预炸鸡腿', 待炸数量)
        
        # 重置员工状态
        if 任务类型 == '当前任务':
            员工['当前任务'] = None
            员工['结束时间'] = None
        else:
            员工['被动任务'] = None
            员工['被动结束时间'] = None
        
        # 检查订单是否完成
        self.检查订单完成状态()
    
    def 添加库存(self, 产品, 数量):
        # 特殊处理冰淇淋（不添加到库存）
        if 产品 == '冰淇淋':
            return
            
        # 设置过期时间    
        过期时间 = self.当前时间 + datetime.timedelta(seconds=self.保存时间.get(产品, 3600))

        # 可乐不占用保温箱空间，直接添加到库存
        if 产品 == '可乐':
            self.库存[产品]['数量'] += 数量
            self.库存[产品]['过期时间'] = 过期时间
            self.添加日志(f"添加{数量}瓶{产品}到饮料柜，有效期至{过期时间.strftime('%H:%M:%S')}")
            return

        # 其他食品需要占用保温箱空间
        保温箱可用空间 = self.设备['保温箱']['容量'] - self.设备['保温箱']['使用中']

        # 计算实际可添加数量
        实际添加数量 = min(数量, 保温箱可用空间)
        if 实际添加数量 > 0:
            self.库存[产品]['数量'] += 实际添加数量
            self.库存[产品]['过期时间'] = 过期时间
            self.设备['保温箱']['使用中'] += 实际添加数量
            self.设备['保温箱']['内容'].append({
                '产品': 产品,
                '数量': 实际添加数量,
                '过期时间': 过期时间
            })
            self.添加日志(f"添加{实际添加数量}个{产品}到保温箱，有效期至{过期时间.strftime('%H:%M:%S')}")

            # 如果实际添加数量小于需求数量，记录空间不足
            if 实际添加数量 < 数量:
                self.添加日志(f"保温箱空间不足，无法添加{数量-实际添加数量}个{产品}")
    
    def 检查订单完成状态(self):
        # 检查所有订单的任务是否都已完成
        所有订单 = self.外卖订单 + self.堂食订单
        已完成订单 = []
        
        for 订单 in 所有订单:
            # 检查该订单是否还有未完成的任务
            还有任务 = False
            for 任务 in self.任务队列:
                if 任务['订单ID'] == 订单['ID']:
                    还有任务 = True
                    break
            
            # 检查前台员工的任务
            for 员工信息 in self.前台员工.values():
                if 员工信息['当前任务'] and 员工信息['当前任务']['订单ID'] == 订单['ID']:
                    还有任务 = True
                    break
            
            # 检查后厨员工的任务
            for 员工信息 in self.后厨员工.values():
                if (员工信息['当前任务'] and 员工信息['当前任务']['订单ID'] == 订单['ID']) or \
                   (员工信息['被动任务'] and 员工信息['被动任务']['订单ID'] == 订单['ID']):
                    还有任务 = True
                    break
            
            if not 还有任务 and 订单['状态'] == '处理中':
                订单['状态'] = '已完成'
                订单['完成时间'] = self.当前时间
                已完成订单.append(订单)
                self.添加日志(f"订单 #{订单['ID']} 已完成处理")
        
        # 移除已完成订单
        for 订单 in 已完成订单:
            if 订单['类型'] == '外卖':
                self.外卖订单.remove(订单)
            else:
                self.堂食订单.remove(订单)
            self.完成订单.append(订单)
    
    def 检查订单超时(self):
        """检查外卖和堂食订单是否超时"""
        当前时间 = self.当前时间
        
        # 检查外卖订单超时
        for 订单 in self.外卖订单:
            等待时间 = (当前时间 - 订单['创建时间']).total_seconds() / 60
            if 等待时间 > 30 and not 订单.get('已报告严重超时', False):
                订单['已报告严重超时'] = True
                self.严重超时订单数 += 1
                self.添加日志(f"⚠️ 外卖订单 #{订单['ID']} 严重超时！(>30分钟)")
            elif 等待时间 > 20 and not 订单.get('已报告超时', False):
                订单['已报告超时'] = True
                self.超时订单数 += 1
                self.添加日志(f"⚠️ 外卖订单 #{订单['ID']} 超时！(>20分钟)")
        
        # 检查堂食订单顾客不满
        需移除堂食订单 = []
        for 订单 in self.堂食订单:
            等待时间 = (当前时间 - 订单['创建时间']).total_seconds() / 60
            
            # 顾客状态从正常变成不高兴
            if 订单['状态'] != '已完成' and 等待时间 > 15 and 订单['顾客状态'] == '正常':
                订单['顾客状态'] = '不高兴'
                self.不高兴顾客数 += 1
                self.添加日志(f"😠 堂食顾客(订单 #{订单['ID']})等待超过15分钟，顾客不高兴!")
            
            # 顾客状态从不高兴变成生气并离开
            elif 订单['状态'] != '已完成' and 等待时间 > 25 and 订单['顾客状态'] == '不高兴':
                订单['顾客状态'] = '生气离开'
                self.生气顾客数 += 1
                self.当前座位使用 -= 订单['座位数']
                self.添加日志(f"😡 堂食顾客(订单 #{订单['ID']})等待超过25分钟，顾客生气离开了，释放{订单['座位数']}个座位!")
                需移除堂食订单.append(订单)
        
        # 移除生气离开的顾客订单
        for 订单 in 需移除堂食订单:
            self.堂食订单.remove(订单)
    
    def 检查座位释放(self):
        # 检查堂食客人是否已离开
        当前时间 = self.当前时间
        需释放的订单 = []
        
        for 订单 in self.堂食订单:
            # 只处理正常完成的订单，生气离开的在别处处理
            if 订单['状态'] == '已完成' and 订单['离开时间'] and 当前时间 >= 订单['离开时间'] and 订单['顾客状态'] != '生气离开':
                需释放的订单.append(订单)
                self.当前座位使用 -= 订单['座位数']
                self.添加日志(f"堂食客人(订单 #{订单['ID']})离开，释放{订单['座位数']}个座位")
        
        # 移除已离开的客人订单
        for 订单 in 需释放的订单:
            self.堂食订单.remove(订单)
    
    def 添加日志(self, 消息):
        时间戳 = self.当前时间.strftime('%H:%M:%S')
        self.日志.append(f"[{时间戳}] {消息}")
        if len(self.日志) > 100:
            self.日志 = self.日志[-100:]
    
    def 检查库存过期(self):
        for 产品, 信息 in self.库存.items():
            if 信息['数量'] > 0 and 信息['过期时间'] and 信息['过期时间'] < self.当前时间:
                self.添加日志(f"{产品}已过期，丢弃{信息['数量']}个")
                
                # 可乐不需要从保温箱中移除
                if 产品 != '可乐':
                    # 从保温箱移除过期食品
                    需移除数量 = 信息['数量']

                    for idx in range(len(self.设备['保温箱']['内容'])-1, -1, -1):
                        内容项 = self.设备['保温箱']['内容'][idx]
                        if 内容项['产品'] == 产品:
                            移除数量 = min(内容项['数量'], 需移除数量)
                            内容项['数量'] -= 移除数量
                            self.设备['保温箱']['使用中'] -= 移除数量
                            需移除数量 -= 移除数量

                            if 内容项['数量'] <= 0:
                                self.设备['保温箱']['内容'].pop(idx)

                            if 需移除数量 <= 0:
                                break

                    信息['数量'] = 0
                    信息['过期时间'] = None
    
    def 智能预测库存(self):
        """基于机器学习预测的订单量智能准备库存"""
        # 每隔5分钟进行一次预测
        当前秒数 = self.当前时间.second + self.当前时间.minute * 60 + self.当前时间.hour * 3600
        if 当前秒数 % self.智能预测周期 != 0:
            return
            
        # 获取未来订单预测
        predicted_orders = self.ml.预测订单高峰(self)
        
        # 如果没有有效预测，则跳过
        if not predicted_orders:
            return
            
        # 根据预测的订单量计算需要的食材数量
        self.添加日志(f"⚙️ 智能系统预测未来30分钟订单量: {predicted_orders:.1f}单")
        
        # 根据预测订单量和食材类型，智能化准备库存
        需求比例 = {
            '汉堡A': 0.4,  # 40%的订单包含汉堡A
            '汉堡B': 0.3,  # 30%的订单包含汉堡B
            '薯条': 0.8,   # 80%的订单包含薯条
            '炸鸡块': 0.6,  # 60%的订单包含炸鸡块
            '炸鸡翅': 0.4,  # 40%的订单包含炸鸡翅
            '炸鸡腿': 0.3,  # 30%的订单包含炸鸡腿
            '可乐': 0.9     # 90%的订单包含可乐
        }
        
        for 产品, 比例 in 需求比例.items():
            # 计算预期需求量
            预期需求 = predicted_orders * 比例
            当前库存 = self.库存[产品]['数量']
            
            # 智能计算需要补充的数量
            if 当前库存 < 预期需求 * 0.6:  # 如果库存低于预期需求的60%，才补充
                需要数量 = int(预期需求 * 1.2 - 当前库存)  # 多准备20%作为缓冲
                
                if 需要数量 > 0:
                    self.添加日志(f"🤖 智能补货: {产品}x{需要数量}")
                    for _ in range(需要数量):
                        for 步骤 in self.产品配方.get(产品, []):
                            任务 = 步骤.copy()
                            任务.update({
                                '订单ID': 0,  # 0表示预制库存任务
                                '状态': '等待中',
                                '产品': 产品,
                                '优先级': 2.3  # 预制库存任务稍高于一般外卖订单
                            })
                            self.任务队列.append(任务)
    
    def 自动准备库存(self):
        # 根据时段准备一定比例的热门产品
        小时 = self.当前时间.hour
        分钟 = self.当前时间.minute
        当前时刻 = 小时 + 分钟/60
        
        # 计算下一个高峰期距离现在的时间
        下个高峰期 = None
        距离分钟 = float('inf')
        
        高峰期时段 = [(11.5, 12.5), (17, 19)]
        for 开始, 结束 in 高峰期时段:
            if 当前时刻 < 开始:
                if 开始 - 当前时刻 < 距离分钟:
                    距离分钟 = 开始 - 当前时刻
                    下个高峰期 = (开始, 结束)
        
        # 如果距离下个高峰期不到90分钟，且当前不在高峰期，则开始准备
        当前在高峰期 = any(开始 <= 当前时刻 < 结束 for 开始, 结束 in 高峰期时段)
        
        if 下个高峰期 and 距离分钟 < 1.5 and not 当前在高峰期:
            # 更新准备数量，增加炸物的预制量
            准备食材 = [
                {'产品': '薯条', '数量': 15},
                {'产品': '炸鸡腿', '数量': 8},
                {'产品': '炸鸡翅', '数量': 8},
                {'产品': '炸鸡块', '数量': 8},
                {'产品': '汉堡A', '数量': 5},
                {'产品': '汉堡B', '数量': 5},
                {'产品': '可乐', '数量': 10}
            ]
            
            for 食材 in 准备食材:
                # 检查当前库存
                当前库存 = self.库存[食材['产品']]['数量']
                if 当前库存 < 食材['数量']:
                    需要数量 = 食材['数量'] - 当前库存
                    self.添加日志(f"准备高峰期库存：{食材['产品']} x {需要数量}")
                    
                    # 添加预制任务
                    for _ in range(需要数量):
                        for 步骤 in self.产品配方.get(食材['产品'], []):
                            任务 = 步骤.copy()
                            任务.update({
                                '订单ID': 0,  # 0表示预制库存
                                '状态': '等待中',
                                '产品': 食材['产品'],
                                '优先级': 2.5  # 预制库存任务优先级最低
                            })
                            self.任务队列.append(任务)
    
    def 模拟一步(self):
        # 更新机器学习模型记录状态
        self.ml.记录状态(self)
        
        # 更新时间
        self.当前时间 += datetime.timedelta(seconds=1)
        
        # 如果超过营业时间，结束模拟
        if self.当前时间 >= self.结束时间:
            return False
        
        # 更新员工数量
        self.更新员工数量()
        
        # 检查任务完成情况 - 前台员工
        for 员工ID, 信息 in self.前台员工.items():
            if 信息['当前任务'] and 信息['结束时间'] and self.当前时间 >= 信息['结束时间']:
                self.完成任务('前台', 员工ID)
        
        # 检查任务完成情况 - 后厨员工主动任务
        for 员工ID, 信息 in self.后厨员工.items():
            if 信息['当前任务'] and 信息['结束时间'] and self.当前时间 >= 信息['结束时间']:
                self.完成任务('后厨', 员工ID)
            
            # 检查被动任务
            if 信息['被动任务'] and 信息['被动结束时间'] and self.当前时间 >= 信息['被动结束时间']:
                self.完成任务('后厨', 员工ID, '被动任务')
        
        # 检查库存过期
        self.检查库存过期()
        
        # 检查订单超时情况
        self.检查订单超时()
        
        # 使用机器学习进行智能库存预测
        self.智能预测库存()
        
        # 分配任务
        self.分配任务()
        
        # 检查座位释放
        self.检查座位释放()
        
        # 生成外卖订单
        if random.random() < self.生成订单概率('外卖'):
            self.添加订单('外卖')
        
        # 生成堂食订单（根据上座率和当前空座情况）
        堂食概率 = self.生成订单概率('堂食')
        if self.当前座位使用 < self.座位数 and random.random() < 堂食概率 * 0.01:  # 降低生成频率
            self.添加订单('堂食')
        
        # 自动准备库存
        if self.当前时间.second == 0:  # 每分钟执行一次
            self.自动准备库存()
        
        return True

    def 生成统计报告(self):
        """生成当前的营业统计报告"""
        已完成外卖 = len([o for o in self.完成订单 if o['类型'] == '外卖'])
        已完成堂食 = len([o for o in self.完成订单 if o['类型'] == '堂食'])
        
        报告 = [
            f"📊 营业统计报告 - {self.当前时间.strftime('%H:%M:%S')}",
            f"===================================",
            f"✅ 已完成订单: {len(self.完成订单)}单 (外卖: {已完成外卖}, 堂食: {已完成堂食})",
            f"⏱️ 待处理订单: {len(self.外卖订单) + len(self.堂食订单)}单 (外卖: {len(self.外卖订单)}, 堂食: {len(self.堂食订单)})",
            f"⚠️ 超时订单数: {self.超时订单数}单 (>20分钟)",
            f"🚨 严重超时: {self.严重超时订单数}单 (>30分钟)",
            f"😠 不高兴顾客: {self.不高兴顾客数}人",
            f"😡 生气离开顾客: {self.生气顾客数}人",
            f"💺 座位使用率: {self.当前座位使用}/{self.座位数} ({int(self.当前座位使用/self.座位数*100) if self.座位数 > 0 else 0}%)"
        ]
        
        # 添加机器学习相关信息
        if self.ml.models_trained:
            报告.append(f"🤖 机器学习模型: 已训练 (数据点: {len(self.ml.history_data)})")
        else:
            报告.append(f"🤖 机器学习模型: 训练中 (数据点: {len(self.ml.history_data)})")
        
        return "\n".join(报告)

# 创建界面
class 餐厅模拟界面:
    def __init__(self):
        self.模拟器 = 餐厅模拟器()
        self.运行中 = False
        self.创建界面()
    
    def 创建界面(self):
        # 控制面板
        self.速度选择 = widgets.Dropdown(
            options=[('1倍速', 1), ('2倍速', 2), ('4倍速', 4), ('8倍速', 8), ('16倍速', 16), ('32倍速', 32)],
            value=1,
            description='模拟速度:',
            style={'description_width': 'initial'}
        )
        
        self.启动按钮 = widgets.ToggleButton(
            value=False,
            description='▶️ 启动模拟',
            button_style='success',
            layout={'width': '150px'}
        )
        
        self.时间标签 = widgets.Label(value="🕒 09:00:00")
        self.统计按钮 = widgets.Button(description="📊 统计报告", button_style='info')
        
        # 输出面板
        self.订单面板 = widgets.Output(layout={'height': 'auto', 'max_width': '400px'})
        self.员工面板 = widgets.Output(layout={'height': 'auto', 'max_width': '400px'})
        self.设备面板 = widgets.Output(layout={'height': 'auto', 'max_width': '400px'})
        self.库存面板 = widgets.Output(layout={'height': 'auto', 'max_width': '400px'})
        self.日志面板 = widgets.Output(layout={'height': '200px', 'max_width': '800px'})
        self.统计面板 = widgets.Output(layout={'height': 'auto', 'max_width': '800px'})
        
        # 布局
        self.控制面板 = widgets.HBox([self.启动按钮, self.速度选择, self.时间标签, self.统计按钮])
        self.状态面板 = widgets.HBox([
            widgets.VBox([widgets.Label(value="📋 活跃订单"), self.订单面板]),
            widgets.VBox([widgets.Label(value="👨‍🍳 员工状态"), self.员工面板]),
            widgets.VBox([widgets.Label(value="🔧 设备状态"), self.设备面板]),
            widgets.VBox([widgets.Label(value="📦 库存状态"), self.库存面板])
        ])
        
        self.日志区 = widgets.VBox([widgets.Label(value="📝 操作日志"), self.日志面板])
        self.统计区 = widgets.VBox([widgets.Label(value="📊 统计报告"), self.统计面板])
        
        self.界面 = widgets.VBox([self.控制面板, self.状态面板, self.日志区, self.统计区])
        
        # 事件处理
        self.启动按钮.observe(self.切换模拟, names='value')
        self.速度选择.observe(self.更新速度, names='value')
        self.统计按钮.on_click(self.显示统计报告)
    
    def 更新界面(self):
        # 更新时间显示
        self.时间标签.value = f"🕒 {self.模拟器.当前时间.strftime('%H:%M:%S')}"
        
        # 更新订单显示
        with self.订单面板:
            clear_output(wait=True)
            print(f"外卖订单: {len(self.模拟器.外卖订单)}个 | 堂食订单: {len(self.模拟器.堂食订单)}个")
            print(f"座位使用: {self.模拟器.当前座位使用}/{self.模拟器.座位数}")
            print("\n活跃外卖订单:")
            for 订单 in self.模拟器.外卖订单[-5:]:
                等待时间 = (self.模拟器.当前时间 - 订单['创建时间']).total_seconds() / 60
                状态图标 = "🟡" if 订单['状态'] == '等待中' else "🟢" if 订单['状态'] == '处理中' else "✅"
                if 等待时间 > 30:
                    状态图标 = "🚨"  # 严重超时
                elif 等待时间 > 20:
                    状态图标 = "⚠️"  # 超时
                
                print(f"{状态图标} #{订单['ID']} {订单['套餐']} ({int(等待时间)}分钟)")
            
            print("\n活跃堂食订单:")
            for 订单 in self.模拟器.堂食订单[-5:]:
                等待时间 = (self.模拟器.当前时间 - 订单['创建时间']).total_seconds() / 60
                
                if 订单['顾客状态'] == '不高兴':
                    状态图标 = "😠"
                elif 订单['顾客状态'] == '生气离开':
                    状态图标 = "😡"
                else:
                    状态图标 = "🟡" if 订单['状态'] == '等待中' else "🟢" if 订单['状态'] == '处理中' else "✅"
                
                print(f"{状态图标} #{订单['ID']} {订单['套餐']} ({int(等待时间)}分钟)")
        
        # 更新员工显示
        with self.员工面板:
            clear_output(wait=True)
            print("🧑‍🔬 前台员工:")
            for 员工ID, 信息 in self.模拟器.前台员工.items():
                if 信息['当前任务'] is None:
                    print(f"前台{员工ID}: 🆓 空闲")
                else:
                    任务 = 信息['当前任务']
                    已用时间 = (self.模拟器.当前时间 - 信息.get('开始时间', self.模拟器.当前时间)).total_seconds()
                    总时间 = 任务['时间']
                    进度 = min(已用时间 / 总时间 * 100, 100)
                    剩余时间 = max(0, 总时间 - 已用时间)
                    
                    print(f"前台{员工ID}: {任务['名称']} ({进度:.0f}%)")
                    print(f"  订单#{任务['订单ID']} | 剩余: {int(剩余时间)}秒")
            
            print("\n👨‍🍳 后厨员工:")
            for 员工ID, 信息 in self.模拟器.后厨员工.items():
                任务列表 = []
                
                # 主要任务
                if 信息['当前任务']:
                    任务 = 信息['当前任务']
                    已用时间 = (self.模拟器.当前时间 - 信息.get('开始时间', self.模拟器.当前时间)).total_seconds()
                    总时间 = 任务['时间']
                    进度 = min(已用时间 / 总时间 * 100, 100)
                    任务列表.append(f"主动: {任务['名称']} ({进度:.0f}%)")
                
                # 被动任务
                if 信息['被动任务']:
                    任务 = 信息['被动任务']
                    已用时间 = (self.模拟器.当前时间 - 信息.get('被动开始时间', self.模拟器.当前时间)).total_seconds()
                    总时间 = 任务['时间']
                    进度 = min(已用时间 / 总时间 * 100, 100)
                    任务列表.append(f"被动: {任务['名称']} ({进度:.0f}%)")
                
                状态 = "🆓 空闲" if not 任务列表 else " + ".join(任务列表)
                print(f"后厨{员工ID}: {状态}")
        
        # 更新设备显示
        with self.设备面板:
            clear_output(wait=True)
            # 炸篮状态
            使用率 = (self.模拟器.炸篮['总数'] - self.模拟器.炸篮['空闲']) / self.模拟器.炸篮['总数'] if self.模拟器.炸篮['总数'] > 0 else 0
            进度条 = "█" * int(使用率 * 10) + "░" * (10 - int(使用率 * 10))
            print(f"炸篮: {进度条} ({self.模拟器.炸篮['总数'] - self.模拟器.炸篮['空闲']}/{self.模拟器.炸篮['总数']})")

            # 其他设备
            for 设备名称, 信息 in self.模拟器.设备.items():
                使用率 = 信息['使用中'] / 信息['容量'] if 信息['容量'] > 0 else 0
                进度条 = "█" * int(使用率 * 10) + "░" * (10 - int(使用率 * 10))
                print(f"{设备名称}: {进度条} ({信息['使用中']}/{信息['容量']})")

                # 添加检查，区分保温箱和其他设备
                if 设备名称 == '保温箱' and 信息['使用中'] > 0:
                    print("  保存内容:")
                    for 内容 in 信息.get('内容', [])[:3]:  # 显示前3个
                        过期时间显示 = 内容.get('过期时间').strftime('%H:%M') if 内容.get('过期时间') else '未知'
                        print(f"  - {内容['产品']}: {内容['数量']}个 (到{过期时间显示})")
                    if len(信息.get('内容', [])) > 3:
                        print(f"  ...共{len(信息.get('内容', []))}项")

                elif 信息.get('任务') and 信息['使用中'] > 0:
                    print("  当前任务:")
                    for 员工类型, 员工ID, 任务 in 信息['任务'][:3]:  # 显示前3个
                        待炸数量 = len(任务.get('待炸数量', [任务['订单ID']]))
                        if 待炸数量 > 1:
                            print(f"  - {员工类型}{员工ID}: {任务['名称']} (批量处理{待炸数量}份)")
                        else:
                            print(f"  - {员工类型}{员工ID}: {任务['名称']} (订单#{任务['订单ID']})")
                    if len(信息.get('任务', [])) > 3:
                        print(f"  ...共{len(信息.get('任务', []))}项")
        
        # 更新库存显示
        with self.库存面板:
            clear_output(wait=True)
            for 产品, 信息 in self.模拟器.库存.items():
                if 产品 != '冰淇淋' and 信息['数量'] > 0 and 信息['过期时间']:
                    剩余时间 = (信息['过期时间'] - self.模拟器.当前时间).total_seconds()
                    if 剩余时间 <= 0:
                        状态 = "已过期"
                    else:
                        小时 = int(剩余时间 // 3600)
                        分钟 = int((剩余时间 % 3600) // 60)
                        状态 = f"剩余{小时}时{分钟}分"
                    print(f"{产品}: {信息['数量']}个 ({状态})")
        
        # 更新日志显示
        with self.日志面板:
            clear_output(wait=True)
            for 日志 in self.模拟器.日志[-20:]:
                print(日志)
    
    def 切换模拟(self, change):
        if change['new']:  # 启动
            self.启动按钮.description = '⏸️ 暂停模拟'
            self.模拟器.暂停 = False
            if not self.运行中:
                self.运行中 = True
                Thread(target=self.模拟线程, daemon=True).start()
        else:  # 暂停
            self.启动按钮.description = '▶️ 继续模拟'
            self.模拟器.暂停 = True
    
    def 更新速度(self, change):
        self.模拟器.速度 = change['new']
    
    def 显示统计报告(self, _):
        with self.统计面板:
            clear_output(wait=True)
            print(self.模拟器.生成统计报告())
    
    def 模拟线程(self):
        while self.运行中:
            if not self.模拟器.暂停:
                # 根据速度执行多步模拟
                for _ in range(self.模拟器.速度):
                    if not self.模拟器.模拟一步():
                        self.运行中 = False
                        self.启动按钮.value = False
                        with self.统计面板:
                            clear_output(wait=True)
                            print(self.模拟器.生成统计报告())
                            print("\n🎉 模拟结束！营业时间已到 22:00")
                        break
                
                # 更新界面
                self.更新界面()
            
            # 控制刷新率
            time.sleep(0.05)

# 启动模拟器
def 启动餐厅模拟():
    界面 = 餐厅模拟界面()
    display(界面.界面)
    界面.更新界面()
    return 界面

# 启动模拟器
餐厅界面 = 启动餐厅模拟()

def get_status():
    """显示餐厅模拟器的当前状态"""
    print("===== 餐厅模拟状态 =====")
    print(f"当前时间: {餐厅界面.模拟器.当前时间.strftime('%H:%M:%S')}")
    print(f"完成订单: {len(餐厅界面.模拟器.完成订单)}个")
    print(f"待处理订单: {len(餐厅界面.模拟器.外卖订单) + len(餐厅界面.模拟器.堂食订单)}个")
    print(f"座位使用率: {餐厅界面.模拟器.当前座位使用}/{餐厅界面.模拟器.座位数}")
    print(f"保温箱使用: {餐厅界面.模拟器.设备['保温箱']['使用中']}/{餐厅界面.模拟器.设备['保温箱']['容量']}")
    
    # 显示保温箱内容
    if 餐厅界面.模拟器.设备['保温箱']['使用中'] > 0:
        print("\n保温箱内容:")
        for 内容 in 餐厅界面.模拟器.设备['保温箱']['内容']:
            过期时间显示 = 内容.get('过期时间').strftime('%H:%M') if 内容.get('过期时间') else '未知'
            print(f"- {内容['产品']}: {内容['数量']}个 (到{过期时间显示})")
    
    # 显示机器学习状态
    print("\n机器学习状态:")
    print(f"数据点数量: {len(餐厅界面.模拟器.ml.history_data)}")
    print(f"模型已训练: {'是' if 餐厅界面.模拟器.ml.models_trained else '否'}")
    
    print("======================")

In [2]:
pip install voila

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
